In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbPipeline

from skfeature.function.information_theoretical_based import MRMR


In [2]:
# Set the decimal format
pd.options.display.float_format = "{:.2f}".format

# Read the data
df = pd.read_csv("diabetes_prediction_dataset.csv")

# Handle duplicates
df = df.drop_duplicates()

# Remove unnecessary values [0.00195%]
df = df[df['gender'] != 'Other']

In [3]:
# Recategorize smoking history
def recategorize_smoking(smoking_status):
    if smoking_status in ['never', 'No Info']:
        return 'non-smoker'
    elif smoking_status == 'current':
        return 'current'
    elif smoking_status in ['ever', 'former', 'not current']:
        return 'past_smoker'

df['smoking_history'] = df['smoking_history'].apply(recategorize_smoking)


In [4]:
# One-hot encoding
def perform_one_hot_encoding(df, column_name):
    dummies = pd.get_dummies(df[column_name], prefix=column_name)
    df = pd.concat([df.drop(column_name, axis=1), dummies], axis=1)
    return df

data = df.copy()
data = perform_one_hot_encoding(data, 'gender')
data = perform_one_hot_encoding(data, 'smoking_history')


In [5]:
# Resampling
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)

# Feature Selection with MRMR
X = data.drop('diabetes', axis=1)
y = data['diabetes']


In [6]:
# Perform MRMR feature selection
num_features_to_select = 11  # Adjust the number of features as needed
feat_idx = MRMR.mrmr(X.values, y.values, n_selected_features=num_features_to_select)
selected_features = X.columns[feat_idx].tolist()

# Select only the MRMR-selected features
X_mrmr = X[selected_features]

# Model Pipeline with Gaussian Naive Bayes (without GridSearchCV)
clf_mrmr = imbPipeline(steps=[
    ('preprocessor', StandardScaler()),
    ('over', over),
    ('under', under),
    ('classifier', GaussianNB())
])


In [7]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_mrmr, y, test_size=0.2, random_state=42)


In [8]:
# Train the model with MRMR-selected features
clf_mrmr.fit(X_train, y_train)

Pipeline(steps=[('preprocessor', StandardScaler()),
                ('over', SMOTE(sampling_strategy=0.1)),
                ('under', RandomUnderSampler(sampling_strategy=0.5)),
                ('classifier', GaussianNB())])

In [9]:
# Predict on the test set
y_pred_mrmr = clf_mrmr.predict(X_test)

# Evaluate the model with MRMR-selected features
print("Model Accuracy with MRMR: ", accuracy_score(y_test, y_pred_mrmr))
print(classification_report(y_test, y_pred_mrmr))

Model Accuracy with MRMR:  0.8869239571413711
              precision    recall  f1-score   support

           0       0.97      0.90      0.94     17525
           1       0.42      0.74      0.54      1701

    accuracy                           0.89     19226
   macro avg       0.70      0.82      0.74     19226
weighted avg       0.92      0.89      0.90     19226

